# Descarga datos esios

## Documentacion:

La nueva web pública de e·sios pone a disposición de todos los usuarios una API para la descarga de información, cuya documentación se encuentra disponible en https://api.esios.ree.es.

Para poder utilizar esta API deberán solicitar un token personal enviando un correo a **consultasios@ree.es**.En mi caso:

token: 9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10

En nuestro caso, la solicitud de datos será para un periodo de tiempo solicitando los datos por código indicador:

https://api.esios.ree.es/indicator/getting_a_specific_indicator_filtering_values_by_a_date_range


## 1. Importamos librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary or bytes to be sent in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      /anaconda3/lib/python3.6/site-packages/requests/api.py
Type:      function


## 2. Definición features

## 3. Solicitud datos

Función para obtener los datos del indicador *ind* solicitado entre dos fechas dadas

In [3]:
def solicita_datos(fecha_ini,fecha_fin,ind):

    headers_={'Accept': 'application/json; application/vnd.esios-api-v1+json',
              'Content-Type': 'application/json',
              'Host': 'api.esios.ree.es',
              'Authorization': 'Token token="9d6bcd627698602fbcd18721cca88b90d0f6e6025963f86be84fa18c87801a10"',
              'Cookie':''
             }

    query_parametros_={'start_date': fecha_ini,
                'end_date':fecha_fin}

    url_='https://api.esios.ree.es/indicators/' + str(ind)
    
    #Hacemos peticion:
    r = requests.get(url_,headers=headers_,params=query_parametros_)
    
    #Comprobamos que trae datos
    assert r.status_code==200
    
    #r.content --> durante el diseño para explorar datos
    
    data=json.loads(r.content)
    
    #type(data)
    #data['indicator']['values'] --> durante el diseño para explorar datos
    
    df_data=pd.DataFrame(data['indicator']['values'])
    
    #df_data.dtypes --> Tenemos que convertir las fechas a formato fecha
    #y crear nuestro df
    
    df=pd.DataFrame()
    df['datetime']=pd.to_datetime([f[:10] + ' ' + f[11:13] + ':00:00' for f in df_data['datetime']]) 
    #df['period']=pd.Series([int(f[11:13])+1 for f in df_data['datetime']]) La crearemos al final
    df['value']=df_data['value']
    
    df=df.set_index('datetime')

    return df

## 4. Agregación datos

In [27]:
historico1=solicita_datos('2018-03-25 00:00:00','2018-03-25 23:59:59',10008)

In [28]:
historico1

,period,value
datetime,,
2018-03-25 00:00:00,1,1484.8
2018-03-25 01:00:00,2,1437.4
2018-03-25 03:00:00,4,1410.5
2018-03-25 04:00:00,5,1402.6
2018-03-25 05:00:00,6,1359.9
2018-03-25 06:00:00,7,1355.9
2018-03-25 07:00:00,8,1293.8
2018-03-25 08:00:00,9,1310.0
2018-03-25 09:00:00,10,1276.0


In [33]:
historico2=solicita_datos('2018-03-26 00:00:00','2018-03-26 23:59:59',10010)

In [34]:
historico2

,period,value
datetime,,
2018-03-26 00:00:00,1,7481.8
2018-03-26 01:00:00,2,7703.7
2018-03-26 02:00:00,3,7769.1
2018-03-26 03:00:00,4,7779.0
2018-03-26 04:00:00,5,7752.6
2018-03-26 05:00:00,6,7840.7
2018-03-26 06:00:00,7,7874.4
2018-03-26 07:00:00,8,8057.3
2018-03-26 08:00:00,9,8122.8


In [35]:
historico_merge=pd.concat([historico1,historico2],axis=1)

In [36]:
historico_merge

,period,value,period,value
datetime,,,,
2018-03-25 00:00:00,1.0,1484.8,NaN,NaN
2018-03-25 01:00:00,2.0,1437.4,NaN,NaN
2018-03-25 03:00:00,4.0,1410.5,NaN,NaN
2018-03-25 04:00:00,5.0,1402.6,NaN,NaN
2018-03-25 05:00:00,6.0,1359.9,NaN,NaN
2018-03-25 06:00:00,7.0,1355.9,NaN,NaN
2018-03-25 07:00:00,8.0,1293.8,NaN,NaN
2018-03-25 08:00:00,9.0,1310.0,NaN,NaN
2018-03-25 09:00:00,10.0,1276.0,NaN,NaN


## 5. Check datos

## 6. Salvar datos